# HabitLedger Demo – Behavioural Money Coach

Welcome to the **HabitLedger** interactive demonstration notebook!

## What is HabitLedger?

HabitLedger is an AI-powered behavioural money coach designed to help users build healthier financial habits through:

- **Behavioural Science Principles**: Uses proven concepts like loss aversion, habit loops, friction reduction, and commitment devices
- **Personalized Interventions**: Suggests specific, actionable strategies tailored to your situation
- **Memory & Tracking**: Remembers your goals, streaks, and struggles across interactions
- **Adaptive Coaching**: Detects patterns and adjusts recommendations over time

## What This Notebook Demonstrates

This notebook showcases:

1. How the agent analyzes user input using keyword-based principle detection
2. How it generates personalized coaching responses with interventions
3. How memory tracks progress across multiple interactions
4. Sample scenarios covering common financial habit challenges

**Note**: This demo uses deterministic keyword matching (no LLM calls yet) to demonstrate the core agent architecture and behaviour analysis logic.